In [1]:
Pkg.checkout("ForwardDiff")

INFO: Checking out ForwardDiff master...
INFO: Pulling ForwardDiff latest master...
INFO: No packages to install, update or remove


In [3]:
Pkg.checkout("ReverseDiff")

INFO: Checking out ReverseDiff master...
INFO: Pulling ReverseDiff latest master...
INFO: No packages to install, update or remove


In [4]:
using Revise

In [6]:
using ReverseDiff
using ReverseDiff: @forward
using ForwardDiff
using DiffResults
using CoordinateTransformations

In [57]:
import Nets2
reload("Nets2")

In [58]:
c = Nets2.Chain(AffineMap(0.9 .* ones(1, 1), 0.1 .+ zeros(1)))

Nets2.Chain{Tuple{CoordinateTransformations.AffineMap{Array{Float64,2},Array{Float64,1}}}}((AffineMap([0.9], [0.1]),))

In [59]:
c([0.5])

1-element Array{Float64,1}:
 0.55

In [60]:
c = Nets2.Chain(
    AffineMap(0.9 .* ones(1, 1), 0.1 .+ zeros(1)),
    Nets2.Affine(fill(1.0, 1, 1), fill(0.5, 1)),
    Nets2.activation(Nets2.elu)
    )

Nets2.Chain{Tuple{CoordinateTransformations.AffineMap{Array{Float64,2},Array{Float64,1}},Nets2.Affine{Array{Float64,2},Array{Float64,1}},Nets2.##1#2{Nets2.#elu}}}((AffineMap([0.9], [0.1]), Nets2.Affine, Nets2.#1))

In [61]:
c([0.5])

1-element Array{Float64,1}:
 1.05

In [62]:
Nets2.params(c)

([1.0], [0.5])

In [63]:
c([1.0])

1-element Array{Float64,1}:
 1.5

In [64]:
params = (fill(1.0, 1, 1), [0.25])

([1.0], [0.25])

In [66]:
c2 = Nets2.with_params(c, params...)

Nets2.Chain{Tuple{CoordinateTransformations.AffineMap{Array{Float64,2},Array{Float64,1}},Nets2.Affine{Array{Float64,2},Array{Float64,1}},Nets2.##1#2{Nets2.#elu}}}((AffineMap([0.9], [0.1]), Nets2.Affine, Nets2.#1))

In [67]:
ReverseDiff.gradient((p...) -> Nets2.with_params(c, p...)([1.0])[1], params)

([1.0], [1.0])

In [68]:
f = (params, x) -> x .+ params
ReverseDiff.jacobian(p -> sum((f(p, [1.0]), ForwardDiff.jacobian(x -> f(p, x), [1.0]))), [1.0])

1×1 Array{Float64,2}:
 1.0

In [71]:
c = Nets2.Chain(
#     AffineMap(0.9 .* ones(1, 1), 0.1 .+ zeros(1)),
    Nets2.Affine(fill(1.0, 1, 1), fill(0.5, 1)),
#     Nets2.activation(Nets2.elu)
    )

Nets2.Chain{Tuple{Nets2.Affine{Array{Float64,2},Array{Float64,1}}}}((Nets2.Affine,))

In [73]:
f = (params, x) -> params[1] * x .+ params[2]
ReverseDiff.jacobian((p...) -> ForwardDiff.jacobian(x -> f(p, x), [1.0]), (fill(1.0, 1, 1), [2.0]))

LoadError: [91mMethodError: Cannot `convert` an object of type ForwardDiff.Dual{ForwardDiff.Tag{##150#152{Tuple{ReverseDiff.TrackedArray{Float64,Float64,2,Array{Float64,2},Array{Float64,2}},ReverseDiff.TrackedArray{Float64,Float64,1,Array{Float64,1},Array{Float64,1}}}},Float64},Float64,1} to an object of type Float64
This may have arisen from a call to the constructor Float64(...),
since type constructors fall back to convert methods.[39m

In [45]:
ct = Nets2.TangentPropagation(c)

Nets2.TangentPropagation{Nets2.Chain{Tuple{CoordinateTransformations.AffineMap{Array{Float64,2},Array{Float64,1}},Nets2.Affine{Array{Float64,2},Array{Float64,1}},Nets2.##1#2{Nets2.#elu}}}}(Nets2.Chain{Tuple{CoordinateTransformations.AffineMap{Array{Float64,2},Array{Float64,1}},Nets2.Affine{Array{Float64,2},Array{Float64,1}},Nets2.##1#2{Nets2.#elu}}}((AffineMap([0.9], [0.1]), Nets2.Affine, Nets2.#1)))

In [46]:
ct([1.0])

([1.5], [0.9])

In [48]:
ReverseDiff.gradient((p...) -> sum(Nets2.with_params(ct, p...)([1.0]))[1], params)

LoadError: [91mMethodError: Cannot `convert` an object of type ForwardDiff.Dual{ForwardDiff.Tag{Nets2.Chain{Tuple{CoordinateTransformations.AffineMap{Array{Float64,2},Array{Float64,1}},Nets2.Affine{ReverseDiff.TrackedArray{Float64,Float64,2,Array{Float64,2},Array{Float64,2}},ReverseDiff.TrackedArray{Float64,Float64,1,Array{Float64,1},Array{Float64,1}}},Nets2.##1#2{Nets2.#elu}}},Float64},Float64,1} to an object of type Float64
This may have arisen from a call to the constructor Float64(...),
since type constructors fall back to convert methods.[39m

In [130]:
net

Nets2.TangentPropagation{Nets2.Chain{Tuple{CoordinateTransformations.AffineMap{Array{Float64,2},Array{Float64,1}},Nets2.Affine{Array{Float64,2},Array{Float64,1}},Nets2.##1#2{Nets2.#elu}}}}(Nets2.Chain{Tuple{CoordinateTransformations.AffineMap{Array{Float64,2},Array{Float64,1}},Nets2.Affine{Array{Float64,2},Array{Float64,1}},Nets2.##1#2{Nets2.#elu}}}((AffineMap([0.9], [0.1]), Nets2.Affine, Nets2.#1)))

In [131]:
Nets2.with_params(net, Nets2.params(net)...)

Nets2.TangentPropagation{Nets2.Chain{Tuple{CoordinateTransformations.AffineMap{Array{Float64,2},Array{Float64,1}},Nets2.Affine{Array{Float64,2},Array{Float64,1}},Nets2.##1#2{Nets2.#elu}}}}(Nets2.Chain{Tuple{CoordinateTransformations.AffineMap{Array{Float64,2},Array{Float64,1}},Nets2.Affine{Array{Float64,2},Array{Float64,1}},Nets2.##1#2{Nets2.#elu}}}((AffineMap([0.9], [0.1]), Nets2.Affine, Nets2.#1)))

In [133]:
Nets2.params(net)

([-3.12548], [-0.313086])

In [136]:
x = [2.0]
yJ = [3.0 1.5]
net = Nets2.TangentPropagation(c)
Nets2.initialize!(randn, Nets2.params(net))

loss(params) = sum(abs2, Nets2.with_params(net, params...)(x) .- yJ)

@show loss(Nets2.params(net))

params = Nets2.params(net)
for i in 1:100
    ∇ = ReverseDiff.gradient(loss, params)
    params = params .- 0.01 .* ∇
end
    
@show loss(Nets2.params(net))

loss(Nets2.params(net)) = 18.17869125618248


LoadError: [91mMethodError: Cannot `convert` an object of type ForwardDiff.Dual{ForwardDiff.Tag{Nets2.Chain{Tuple{CoordinateTransformations.AffineMap{Array{Float64,2},Array{Float64,1}},Nets2.Affine{ReverseDiff.TrackedArray{Float64,Float64,2,Array{Float64,2},Array{Float64,2}},ReverseDiff.TrackedArray{Float64,Float64,1,Array{Float64,1},Array{Float64,1}}},Nets2.##1#2{Nets2.#elu}}},Float64},Float64,1} to an object of type Float64
This may have arisen from a call to the constructor Float64(...),
since type constructors fall back to convert methods.[39m